###Creating month wise partitioned parquet file from csv

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

schema = StructType([
    StructField("order_id", StringType(), True),
    StructField("order_date", DateType(), True),
    StructField("order_time", StringType(), True)])

orders = spark.read.format("csv").option("header",True).schema(schema).load("abfss://raw@enterprisedatalake12.dfs.core.windows.net/orders.csv")
# orders.show()
orders_concat =orders.withColumn("Order_Hour",substring(orders.order_time,0,2).cast("Integer")).withColumn("Order_Month", substring(orders.order_date,6,2).cast("Integer"))
orders_concat.show()
orders_concat.write.format('parquet').partitionBy("Order_Month").mode("overwrite").save("abfss://raw@enterprisedatalake12.dfs.core.windows.net/silver/orders")



In [0]:
# df = spark.read.format("parquet").load("dbfs:/FileStore/orders")
# df.display()

###Create Permanent Delta table from CSV

In [0]:
location = "/FileStore/order_details.csv"
file_type = "csv"

infer_schema = "True"
first_row_is_header = "True"
delimiter = ","

delta_df = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(location)


permanent_table_name = "order_details_delta"

delta_df.write.format("delta").saveAsTable(permanent_table_name)

In [0]:
# dbutils.fs.rm('dbfs:/FileStore/orders',recurse=True )
# dbutils.fs.rm('dbfs:/user/hive/warehouse/order_details_delta',recurse=True )
# dbutils.fs.rm('dbfs:/user/hive/warehouse/order_details_delta_',recurse=True )
# dbutils.fs.rm('dbfs:/FileStore/tables/output/Pizza_Data_Analysis',recurse=True )

